In [ ]:
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding


import matplotlib.pyplot as plt # visuals
import tensorflow as tf # tensorflow

from tensorflow.keras.losses import sparse_categorical_crossentropy # loss function
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, GRU, Dense, Dropout
import numpy as np


In [ ]:

path = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
data = open(path, 'r').read()


# Tokenization and Padding

In [ ]:
# integer encode text
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
encoded_data= tokenizer.texts_to_sequences([data])[0]
encoded_data

[88,
 269,
 139,
 35,
 969,
 143,
 668,
 127,
 15,
 102,
 33,
 102,
 102,
 88,
 269,
 6,
 40,
 33,
 1268,
 350,
 3,
 199,
 63,
 3,
 3332,
 33,
 1268,
 1268,
 88,
 269,
 88,
 6,
 91,
 1141,
 231,
 11,
 2274,
 580,
 3,
 1,
 305,
 33,
 35,
 2555,
 35,
 2555,
 88,
 269,
 70,
 78,
 502,
 26,
 2,
 369,
 23,
 1340,
 54,
 39,
 162,
 2275,
 642,
 8,
 3972,
 33,
 32,
 52,
 2875,
 883,
 70,
 16,
 17,
 164,
 148,
 148,
 166,
 269,
 72,
 219,
 46,
 612,
 88,
 269,
 35,
 40,
 5005,
 173,
 612,
 1,
 1895,
 46,
 28,
 1269,
 5006,
 45,
 56,
 3973,
 78,
 38,
 58,
 56,
 581,
 78,
 20,
 1,
 5007,
 335,
 16,
 76,
 1896,
 35,
 258,
 1011,
 58,
 6929,
 78,
 6930,
 20,
 58,
 133,
 35,
 40,
 99,
 266,
 1,
 3974,
 9,
 6931,
 78,
 1,
 2276,
 5,
 39,
 1516,
 11,
 24,
 87,
 6932,
 3,
 6933,
 64,
 3975,
 39,
 2876,
 11,
 8,
 1341,
 3,
 65,
 70,
 78,
 630,
 21,
 14,
 39,
 3976,
 250,
 35,
 654,
 6934,
 13,
 1,
 424,
 91,
 4,
 102,
 21,
 10,
 3977,
 13,
 2277,
 12,
 10,
 3333,
 13,
 630,
 166,
 269,
 56,
 6,
 969,
 3

In [ ]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

maxlen = 5  # Example length, adjust as needed
X_padded = pad_sequences([encoded_data], maxlen=maxlen, padding='pre')


In [ ]:
# determine the vocabulary size
vocab_size = len(tokenizer.word_index) + 1  # 0 is reserved for padding so that's why we added 1

print('Vocabulary Size: %d' % vocab_size)



Vocabulary Size: 12633


# Next, we need to create sequences of words to fit the model with one word as input and one word as output.


In [ ]:
# create word -> word sequences
sequences = list()
for i in range(1, len(encoded_data)):
	sequence = encoded_data[i-1:i+1]
	sequences.append(sequence)

# step_size = 2
# sequences = list()
# for i in range(0, len(X_padded)-1, step_size):
#     sequence = X_padded[i:i+2]
#     sequences.append(sequence)

# sequences = np.array(sequences)
# X, y = sequences[:, 0], sequences[:, 1]
# print('Total Sequences: %d' % len(sequences))
# # split into X and y elements


# Running this piece shows the total number of input-output pairs to train the network

In [ ]:
sequences
#sequences[:5] # [input, output]

[[88, 269],
 [269, 139],
 [139, 35],
 [35, 969],
 [969, 143],
 [143, 668],
 [668, 127],
 [127, 15],
 [15, 102],
 [102, 33],
 [33, 102],
 [102, 102],
 [102, 88],
 [88, 269],
 [269, 6],
 [6, 40],
 [40, 33],
 [33, 1268],
 [1268, 350],
 [350, 3],
 [3, 199],
 [199, 63],
 [63, 3],
 [3, 3332],
 [3332, 33],
 [33, 1268],
 [1268, 1268],
 [1268, 88],
 [88, 269],
 [269, 88],
 [88, 6],
 [6, 91],
 [91, 1141],
 [1141, 231],
 [231, 11],
 [11, 2274],
 [2274, 580],
 [580, 3],
 [3, 1],
 [1, 305],
 [305, 33],
 [33, 35],
 [35, 2555],
 [2555, 35],
 [35, 2555],
 [2555, 88],
 [88, 269],
 [269, 70],
 [70, 78],
 [78, 502],
 [502, 26],
 [26, 2],
 [2, 369],
 [369, 23],
 [23, 1340],
 [1340, 54],
 [54, 39],
 [39, 162],
 [162, 2275],
 [2275, 642],
 [642, 8],
 [8, 3972],
 [3972, 33],
 [33, 32],
 [32, 52],
 [52, 2875],
 [2875, 883],
 [883, 70],
 [70, 16],
 [16, 17],
 [17, 164],
 [164, 148],
 [148, 148],
 [148, 166],
 [166, 269],
 [269, 72],
 [72, 219],
 [219, 46],
 [46, 612],
 [612, 88],
 [88, 269],
 [269, 35],
 [35, 

In [ ]:
print(len(sequences))


204088


In [ ]:
# Calculate the index to split the array in half
split_index = len(sequences) // 4

# Keep only the first half of the sequences
sequences_reduced = sequences[:split_index]

print('Original Sequences: %d' % len(sequences))
print('Reduced Sequences: %d' % len(sequences_reduced))

Original Sequences: 204088
Reduced Sequences: 51022


# We can then split the sequences into input (X) and output elements (y)

In [ ]:
sequences_reduced = array(sequences_reduced)
X, y = sequences_reduced[:,0],sequences_reduced[:,1]



In [ ]:
X[:5]

array([ 88, 269, 139,  35, 969])

In [ ]:
y[:5]

array([269, 139,  35, 969, 143])

In [ ]:
# one hot encode outputs
y = to_categorical(y, num_classes=vocab_size)
# define model
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

## Training LSTM Case 1 (50 lstm units)




In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=1))
model.add(LSTM(50))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1, 10)             126330    
                                                                 
 lstm_4 (LSTM)               (None, 50)                12200     
                                                                 
 dense_3 (Dense)             (None, 12633)             644283    
                                                                 
Total params: 782813 (2.99 MB)
Trainable params: 782813 (2.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
# compile network
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:
# fit network
# model.fit(X, y, epochs=3, batch_size=32)

model.fit(X_train, y_train, epochs=4, batch_size=32)

# Evaluate on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')


Epoch 1/4
1276/1276 [==============================] - 34s 24ms/step - loss: 7.1864 - accuracy: 0.0334
Epoch 2/4
1276/1276 [==============================] - 29s 22ms/step - loss: 6.5121 - accuracy: 0.0337
Epoch 3/4
1276/1276 [==============================] - 29s 23ms/step - loss: 6.4421 - accuracy: 0.0337
Epoch 4/4
319/319 [==============================] - 5s 13ms/step - loss: 6.8173 - accuracy: 0.0396
Test Loss: 6.817288398742676, Test Accuracy: 0.03958843648433685


In [ ]:
# enter_text=provide, n_pred=3
# # in_text and result =provide
# in_text = provide
# result = provide

enter_text, n_pred = "provide", 3
in_text = enter_text
result = enter_text



In [ ]:

def generate_seq(model, tokenizer, enter_text, n_pred):
    in_text = enter_text
    result = in_text

    for _ in range(n_pred):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        encoded = np.array(encoded)

        # predict a word in the vocabulary
        yhat = model.predict(encoded)

        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == np.argmax(yhat):
                out_word = word
                break

        # append to input
        in_text += ' ' + out_word
        result += ' ' + out_word

    return result




In [ ]:
# evaluate
print(generate_seq(model, tokenizer, 'thou', 2))

1/1 [==============================] - 0s 23ms/step
thou lord 


##Model with more hidden layers

In [ ]:

model2 = Sequential()
model2.add(Embedding(vocab_size, 8, input_length=1))

model2.add(LSTM(50, return_sequences=True))
model2.add(LSTM(50))
model2.add(Dense(vocab_size, activation='softmax'))
print(model2.summary())




Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 1, 8)              101064    
                                                                 
 lstm_5 (LSTM)               (None, 1, 50)             11800     
                                                                 
 lstm_6 (LSTM)               (None, 50)                20200     
                                                                 
 dense_4 (Dense)             (None, 12633)             644283    
                                                                 
Total params: 777347 (2.97 MB)
Trainable params: 777347 (2.97 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:

model2.fit(X_train, y_train, epochs=4, batch_size=32)

# Evaluate on the test set
loss, accuracy = model2.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/4
1276/1276 [==============================] - 37s 24ms/step - loss: 7.0334 - accuracy: 0.0331
Epoch 2/4
1276/1276 [==============================] - 31s 24ms/step - loss: 6.5100 - accuracy: 0.0337
Epoch 3/4
1276/1276 [==============================] - 30s 23ms/step - loss: 6.4470 - accuracy: 0.0337
Epoch 4/4
319/319 [==============================] - 5s 14ms/step - loss: 6.9210 - accuracy: 0.0366
Test Loss: 6.920963287353516, Test Accuracy: 0.0366487018764019


In [ ]:
# evaluate
print(generate_seq(model2, tokenizer, 'thou', 2))

1/1 [==============================] - 0s 23ms/step
thou and 


## Increase number of LSTM units from 50->70

In [ ]:


model3 = Sequential()
model3.add(Embedding(vocab_size, 10, input_length=1))

model3.add(LSTM(70))
model3.add(Dense(vocab_size, activation='softmax'))
print(model3.summary())




Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 1, 10)             126330    
                                                                 
 lstm_7 (LSTM)               (None, 70)                22680     
                                                                 
 dense_5 (Dense)             (None, 12633)             896943    
                                                                 
Total params: 1045953 (3.99 MB)
Trainable params: 1045953 (3.99 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [ ]:
model3.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



In [ ]:

model3.fit(X_train, y_train, epochs=4, batch_size=32)
# Evaluate on the test set
loss, accuracy = model3.evaluate(X_test, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

Epoch 1/4
1276/1276 [==============================] - 38s 28ms/step - loss: 7.1392 - accuracy: 0.0334
Epoch 2/4
1276/1276 [==============================] - 34s 26ms/step - loss: 6.5078 - accuracy: 0.0337
Epoch 3/4
1276/1276 [==============================] - 35s 28ms/step - loss: 6.4297 - accuracy: 0.0338
Epoch 4/4
319/319 [==============================] - 5s 13ms/step - loss: 6.7716 - accuracy: 0.0388
Test Loss: 6.771580219268799, Test Accuracy: 0.03880450874567032


In [ ]:
print(generate_seq(model3, tokenizer, 'thou', 2))

1/1 [==============================] - 0s 23ms/step
thou have 
